---
output-file: individual-unittests.html
title: DHPCTIndividual unittests

---


In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
#| default_exp individual_unittests


In [ ]:
#| export
import numpy as np
import unittest
from dpct.individual import DHPCTIndividual

## PCT Single Level Comparison Test

This test compares the output of a DPCT individual with one level and one column against a mock PCT implementation, with all weights set to 1.

In [ ]:
#| export
class MockPCT:
    """
    Mock PCT implementation for comparison with DPCT.
    Implements a single level PCT control system with weights set to 1.
    """
    def __init__(self, observation_dim, action_dim):
        self.observation_dim = observation_dim
        self.action_dim = action_dim
        # All weights set to 1 as per requirement
        self.perception_weights = np.ones((observation_dim, 1))
        self.output_weights = np.ones((1, 1))
        self.action_weights = np.ones((1, action_dim))
    
    def run_step(self, observation, reference):
        """
        Run one step of PCT control.
        
        Args:
            observation: Input observations (numpy array)
            reference: Reference signal (numpy array)
        
        Returns:
            action: Control action
            error: Control error
        """
        # Perception: weighted sum of observations
        perception = np.dot(observation, self.perception_weights)
        
        # Comparator: reference - perception (error signal)
        error = reference - perception
        
        # Output: element-wise multiplication of error and weighted error
        # This matches DPCT's: Multiply([comparator, Dense(comparator)])
        # Which is: comparator * (comparator * weight) = comparator^2 when weight=1
        weighted_error = np.dot(error, self.output_weights)
        output = error * weighted_error  # Element-wise multiplication
        
        # Action: weighted output
        action = np.dot(output, self.action_weights)
        
        return action.flatten(), error.flatten()

In [ ]:
#| export
def test_pct_single_level_comparison():
    """
    Test comparing DPCT and mock PCT with single level, one column, weights set to 1.
    """
    print("Starting PCT single level comparison test...")
    
    # Create DPCT individual with one level and one column
    dpct_individual = DHPCTIndividual(
        env_name='CartPole-v1',
        levels=[1],  # One level with one column
        activation_funcs={0: 'linear'},  # Linear activation
        weight_types={0: 'float'}
    )
    
    # Compile the DPCT individual
    dpct_individual.compile()
    
    # Set all weights to 1 in DPCT model
    for layer in dpct_individual.model.layers:
        weights = layer.get_weights()
        if weights:  # Only update layers that have weights
            new_weights = [np.ones_like(w) for w in weights]
            layer.set_weights(new_weights)
    
    print("DPCT individual created and weights set to 1")
    
    # Create mock PCT with matching dimensions
    # CartPole has 4 observations, 1 action (discrete but we'll treat as continuous)
    mock_pct = MockPCT(observation_dim=4, action_dim=1)
    print("Mock PCT created with weights set to 1")
    
    # Test with a sample observation and reference
    test_observation = np.array([0.1, 0.2, 0.3, 0.4])  # Sample observation
    test_reference = np.array([0.0])  # Reference signal (zero for simplicity)
    
    # Run DPCT
    obs_input = np.expand_dims(test_observation, axis=0)  # Batch dimension
    ref_input = np.expand_dims(test_reference, axis=0)  # Batch dimension
    dpct_outputs = dpct_individual.model([obs_input, ref_input])
    dpct_action = dpct_outputs[0].numpy().squeeze()
    dpct_error = dpct_outputs[1].numpy().squeeze()
    
    print(f"DPCT - Action: {dpct_action}, Error: {dpct_error}")
    
    # Run mock PCT
    pct_action, pct_error = mock_pct.run_step(test_observation, test_reference)
    
    print(f"Mock PCT - Action: {pct_action}, Error: {pct_error}")
    
    # Compare outputs
    action_diff = np.abs(dpct_action - pct_action)
    error_diff = np.abs(dpct_error - pct_error)
    
    print(f"Action difference: {action_diff}")
    print(f"Error difference: {error_diff}")
    
    # Assert that outputs are reasonably close (allowing for small numerical differences)
    tolerance = 1e-5
    
    # For a single level PCT with linear activation and weights=1:
    # - Perception = sum of observations = 0.1 + 0.2 + 0.3 + 0.4 = 1.0
    # - Error = reference - perception = 0.0 - 1.0 = -1.0
    # - Action should be related to this error
    
    expected_perception = np.sum(test_observation)
    expected_error = test_reference[0] - expected_perception
    
    print(f"Expected perception: {expected_perception}")
    print(f"Expected error: {expected_error}")
    
    # Verify that both implementations produce the same error
    assert np.allclose(dpct_error, pct_error, atol=tolerance), f"Errors don't match: DPCT={dpct_error}, PCT={pct_error}"
    
    # Verify that both implementations produce the same action
    assert np.allclose(dpct_action, pct_action, atol=tolerance), f"Actions don't match: DPCT={dpct_action}, PCT={pct_action}"
    
    # Verify that the error matches expected calculation
    assert np.allclose(pct_error, expected_error, atol=tolerance), f"Error calculation incorrect: got={pct_error}, expected={expected_error}"
    
    print("✓ Test passed: DPCT and PCT outputs match with single level, one column, weights=1")
    
    return True

In [ ]:
# Run the test
test_pct_single_level_comparison()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()